# Day1 · CME（资本市场预期）教学脚手架 · Part 1

> 目标：用**极简可运行**的Notebook，完成CME层的**数据模型与函数骨架**搭建，
为后续 MVO/BL/TAA 链路提供 μ（期望收益）与口径说明的容器。

只包含 **Part 1（Setup & Data Model + Intuition + Skeleton）**，以避免Cursor超长文件问题。


## 本Notebook结构（Part 1）
1. 环境与依赖导入（轻量）
2. 统一数据模型：`MarketAssumptions / RiskModel / Constraints / BLViews / TAASignals`
3. CME直觉与口径注释（债券=YTM；股票=D+g+Δ估值；黄金/商品=通胀/实利β）
4. `estimate_mu_cme()` 函数骨架（**返回占位与notes**，不硬编码主观数值）
5. 快速Smoke Test（创建一个最小的 `market_indicators` DataFrame 并调用函数）

> **提示**：本Part不做任何“数值定参”，只做**容器与口径**。后续Part将补全：
- Part 2：场景（base/bear/bull）生成器
- Part 3：Σ估计（历史协方差 vs 稳健估计）
- Part 4：可视化与导出（雷达图/表格）


In [ ]:
# 1) 环境与依赖导入（轻量）
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd

AssetCode = str  # 例如: "cash","short_bond","credit","equity_div","equity_growth","gold","reits","commodity"
Scenario = str   # 例如: "base","bear","bull"


In [ ]:
# 2) 统一数据模型（与后续MVO/BL/TAA对齐）
@dataclass
class MarketAssumptions:
    mu: Dict[AssetCode, Optional[float]]                       # 期望收益(年化)；本Part先留None作为占位
    mu_scn: Dict[Scenario, Dict[AssetCode, Optional[float]]]  # 分场景μ（本Part只做空壳）
    notes: Dict[AssetCode, str]                                # 口径说明（债=YTM；股=股息+增速+估值回归；金=通胀/实利β）

@dataclass
class RiskModel:
    Sigma: np.ndarray                     # 协方差矩阵(资产×资产)
    assets: List[AssetCode]               # 顺序需与 Sigma 对齐
    vol_annual: Dict[AssetCode, float]    # 年化波动(可选)
    corr: Optional[np.ndarray] = None     # 相关性矩阵(可选，用于展示/校验)

@dataclass
class Constraints:
    bounds: Dict[AssetCode, Tuple[float, float]]    # 权重上下限
    vol_cap: Optional[float] = None                 # 组合波动上限
    liq_floor_7d: Optional[float] = None            # 7日可变现占比下限
    budget_eq_1: bool = True                        # ∑w=1

@dataclass
class BLViews:
    pi: Optional[np.ndarray] = None  # 市场隐含回报(先验), shape=(n,1)
    P: Optional[np.ndarray] = None   # 观点矩阵, shape=(m,n)
    q: Optional[np.ndarray] = None   # 观点向量, shape=(m,1)
    Omega: Optional[np.ndarray] = None   # 观点协方差, shape=(m,m)
    tau: float = 0.05

@dataclass
class TAASignals:
    s: np.ndarray                # 标准化信号向量, shape=(k,1)
    B: np.ndarray                # 灵敏度矩阵(资产×信号), shape=(n,k)
    tactical_cap: float = 0.05   # 战术偏移上限(绝对值)
    rebalance_band: float = 0.20 # 与目标偏离±20%触发再平衡


## 3) CME直觉与口径注释（业务语义）
- **债券**：μ ≈ 当前到期收益率（YTM）口径；必要时考虑违约损失/费用口径。
- **股票**：μ ≈ 股息率 (D/P) + 长期盈利增速 (g) + 估值回归（Δ估值，多为均值回归项）。
- **黄金/商品**：μ ≈ 通胀 / 实际利率 等宏观变量的 β 敏感度口径（由回归估计β）。
- **现金**：μ ≈ 近似无风险利率口径。

> **本Part仅构造容器与说明，不写死任何μ数值。** 后续Part通过场景器与数据映射补全。

In [ ]:
def estimate_mu_cme(
    market_indicators: pd.DataFrame,
    assets: List[AssetCode],
    scenario_labels: List[Scenario] = ["base","bear","bull"],
) -> MarketAssumptions:
    """
    构造各资产的 μ (期望收益) 场景占位；本Part不在此处硬编码主观数值。

    参数
    -----
    market_indicators: 包含公共口径数据(如YTM, dividend_yield, earnings_growth, inflation等)
    assets: 资产代码列表（需与后续RiskModel保持一致）
    scenario_labels: 场景标签列表（默认 base / bear / bull）

    返回
    -----
    MarketAssumptions(mu, mu_scn, notes)
    """
    mu = {a: None for a in assets}  # base场景占位
    mu_scn = {scn: {a: None for a in assets} for scn in scenario_labels}
    notes = {
        "short_bond":"YTM口径(到期收益率)作为基线",
        "credit":"信用溢价在YTM上体现, 可考虑风险/费用口径",
        "equity_div":"股息率+盈利增速+估值回归(中性)",
        "equity_growth":"同上, 但成长假设不同",
        "gold":"通胀/实际利率敏感度(β)口径",
        "reits":"分红率与租金增长口径",
        "commodity":"库存/通胀/美元相关口径",
        "cash":"近似无风险利率口径",
    }
    return MarketAssumptions(mu=mu, mu_scn=mu_scn, notes=notes)


In [ ]:
# 5) 快速 Smoke Test（最小可运行）
assets = ["cash","short_bond","credit","equity_div","equity_growth","gold","reits","commodity"]
market_indicators = pd.DataFrame({
    # 仅作为列占位示例；真实数值/口径将在Part 2填充
    "risk_free_rate": [None],
    "ytm_bond": [None],
    "dividend_yield": [None],
    "earnings_growth": [None],
    "valuation_reversion": [None],
    "inflation_expectation": [None],
    "real_rate": [None],
})

assumptions = estimate_mu_cme(market_indicators, assets)
print("μ keys:", list(assumptions.mu.keys()))
print("场景:", list(assumptions.mu_scn.keys()))
print("notes[gold]:", assumptions.notes.get("gold"))


### ✅ TODO（进入 Part 2 前的检查点）
1. **是否需要调整资产清单**（与后端产品库对齐）？
2. 是否要新增**口径字段**，例如：股息口径（TTM/一致预期）、估值回归期（3/5/10年）？
3. `market_indicators` 是否应扩展为 **多指标多期**（便于做滚动场景）？
4. 确认与 **RiskModel.Sigma** 的资产顺序一致性约束。

> 通过以上检查后，可在 **Part 2** 中实现：
- 依据不同口径表与经验参数，生成 base/bear/bull 的 μ 数值；
- 将 μ_scn 写入 `MarketAssumptions` 并输出为结构化 JSON/CSV 供后续 MVO 使用。